In [1]:
import time 
import serial
import modbus_crc

In [2]:
#初始化串口
def main():
    global s
    com=input("请输入串口号：")
    port=eval(input("请输入波特率："))
    s=serial.Serial(com,port)

#创建添加ModbusCRC16校验码的函数
def add_crc(data):
    data_to_send=data+bytes(modbus_crc.crc16(data))
    return data_to_send

#改变速度
def change_speed():
    speed=eval(input("如果选择默认速度则输入0,否则请输入设置速度:"))
    if not speed:
        s.flushInput()
        data_speed=[0x01,0x06,0x00,0x05,0x00,0xFF,0xD9,0x8B]
        s.write(bytes(data_speed))
        time.sleep(1)
    else:
        s.flushInput()
        data_speed=[0x01,0x06,0x00,0x05]
        speed=int(speed)
        data_speed.append(speed//256)
        data_speed.append(speed%256)
        data_speed=bytes(data_speed)
        data_speed_to_send=add_crc(data_speed)
        s.write(data_speed_to_send)
        time.sleep(1)

#定义一个将距离换算为对应脉冲的函数
def transform_distance(distance,data_distance):
    k=1
    peak_number=int(distance/k)
    data_distance.append(peak_number//256)
    data_distance.append(peak_number%256)
    data_distance=bytes(data_distance)
    data_distance_to_send=add_crc(data_distance)
    return data_distance_to_send

#改变距离
def change_distance():
    distance=int(input("请输入移动距离:"))
    mode_selection=input("请输入正转或反转：")
    #data_distance=[0x01,0x06,0x00,0x07,0x00,0xFF,0x78,0x4B]
    data_distance=[0x01,0x06,0x00,0x07]
    
    if mode_selection=="正转":
        s.flushInput()
        data_zhengzhuan=[0x01,0x06,0x00,0x00,0x00,0x01,0x48,0x0A]
        s.write(bytes(data_zhengzhuan))
        time.sleep(1)
        data_distance_to_send=transform_distance(distance,data_distance)
        s.write(data_distance_to_send)
        
    else:
        s.flushInput()
        data_fanzhuan=[0x01,0x06,0x00,0x01,0x00,0x01,0x19,0xCA]
        s.write(bytes(data_fanzhuan))
        time.sleep(1)
        data_distance_to_send=transform_distance(distance,data_distance)
        s.write(data_distance_to_send)

#停止运动
def stop_move():
    yes_or_no=eval(input("如果要停止直线模组前进:请输入1;否则,请输入0"))
    if yes_or_no:
        data_close=[0x01,0x06,0x00,0x02,0x00,0x01,0xE9,0xCA]

In [3]:
#初始化串口
main()

In [4]:
#设置速度
change_speed()

In [5]:
#设置距离
change_distance()

In [6]:
#停止运动
stop_move()